In [6]:
import numpy as np 
import pandas as pd

df = pd.read_csv('SolarPrediction.csv')

df['month'] = pd.to_datetime(df['Data']).dt.month
df['day'] = pd.to_datetime(df['Data']).dt.day

df['hour'] = pd.to_datetime(df['Time']).dt.hour
df['minute'] = pd.to_datetime(df['Time']).dt.minute
df['second'] = pd.to_datetime(df['Time']).dt.second

df['rise_hour'] = pd.to_datetime(df['TimeSunRise']).dt.hour
df['rise_minute'] = pd.to_datetime(df['TimeSunRise']).dt.minute

df['set_hour'] = pd.to_datetime(df['TimeSunSet']).dt.hour
df['set_minute'] = pd.to_datetime(df['TimeSunSet']).dt.minute

df.drop(['UNIXTime', 'Data', 'Time', 'TimeSunRise', 'TimeSunSet'], axis = 1, inplace = True)

def get_day(row):
    if row['rise_hour'] < row['hour'] < row['set_hour']:
        return 1
    else:
        return 0
df['day'] = df.apply(get_day, axis=1) # создание нового признака, когда время наблюдения в период с восхода до захода солнца

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import pickle

X = df[['Temperature', 'Pressure','Humidity', 'WindDirection(Degrees)', 'Speed', 'hour']]
y = np.log(df['Radiation'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sts = StandardScaler()
X_train = sts.fit_transform(X_train)
X_test = sts.transform(X_test)

from catboost import CatBoostRegressor
cbr_model = CatBoostRegressor(colsample_bylevel=1.0, learning_rate=0.1, max_depth=10, n_estimators=200, reg_lambda=3.0, subsample=1.0)
cbr_model.fit(X_train, y_train)

with open('cbr_model.pkl', 'wb') as model_file:
    pickle.dump(cbr_model, model_file)

0:	learn: 2.5113726	total: 62.8ms	remaining: 12.5s
1:	learn: 2.2835248	total: 73.6ms	remaining: 7.28s
2:	learn: 2.0824467	total: 76.6ms	remaining: 5.03s
3:	learn: 1.8972886	total: 89.7ms	remaining: 4.39s
4:	learn: 1.7294519	total: 99.3ms	remaining: 3.87s
5:	learn: 1.5821039	total: 108ms	remaining: 3.51s
6:	learn: 1.4479223	total: 113ms	remaining: 3.12s
7:	learn: 1.3306312	total: 120ms	remaining: 2.88s
8:	learn: 1.2262571	total: 125ms	remaining: 2.65s
9:	learn: 1.1316009	total: 129ms	remaining: 2.46s
10:	learn: 1.0490114	total: 135ms	remaining: 2.32s
11:	learn: 0.9785141	total: 139ms	remaining: 2.18s
12:	learn: 0.9141570	total: 143ms	remaining: 2.06s
13:	learn: 0.8580452	total: 148ms	remaining: 1.97s
14:	learn: 0.8111373	total: 153ms	remaining: 1.89s
15:	learn: 0.7669053	total: 157ms	remaining: 1.81s
16:	learn: 0.7290630	total: 162ms	remaining: 1.75s
17:	learn: 0.6965697	total: 168ms	remaining: 1.7s
18:	learn: 0.6693633	total: 173ms	remaining: 1.65s
19:	learn: 0.6455924	total: 181ms	rem